In [ ]:
import io
import zipfile

from glob import glob

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import pydicom
from pydicom.filebase import DicomBytesIO

from pymedphys.coll import calc_mu_density, get_grid

In [ ]:
zip_filepath = glob('../../../tests/data/logfiles/Trilogy/*')[0]

In [ ]:
with open(zip_filepath, 'rb') as input_file:
    data = io.BytesIO(input_file.read())

In [ ]:
data_zip = zipfile.ZipFile(data)

In [ ]:
data_zip.namelist()

In [ ]:
# pd.read_csv?

In [ ]:
with data_zip.open('VMAT_TS4_2-field_H&N/A20190110141052_Anonymous.dlg') as fp:
    dfA = pd.read_csv(fp, skiprows=6, header=None)
    
with data_zip.open('VMAT_TS4_2-field_H&N/B20190110141052_Anonymous.dlg') as fp:
    dfB = pd.read_csv(fp, skiprows=6, header=None)

In [ ]:
# dfA

In [ ]:
dynalog_arrayA = dfA.values
dynalog_arrayB = dfB.values
# dynalog_arrayA[-1,:]

In [ ]:
Y1A = dynalog_arrayA[:,8]
Y1B = dynalog_arrayB[:,8]

assert np.all(Y1A == Y1B)

Y2A = dynalog_arrayA[:,9]
Y2B = dynalog_arrayB[:,9]

assert np.all(Y2A == Y2B)


jaw = np.array([Y1A, Y2A]).T
np.shape(jaw)

# jaw

In [ ]:
# mlc_1 = dynalog_array[:,14::4]
mlc_A = dynalog_arrayA[:,15::4] / 100
mlc_B = dynalog_arrayB[:,15::4] / 100
# mlc_3 = dynalog_array[:,16::4]
# mlc_4 = dynalog_array[:,17::4]

## Couldn't find total MU with Logfile

In [ ]:
### COULDN'T FIND TOTAL MU WITHIN LOGFILE ###
assumed_total_mu = 295

In [ ]:
muA = dynalog_arrayA[:,0] / np.max(dynalog_arrayA[:,0]) * assumed_total_mu
muB = dynalog_arrayB[:,0] / np.max(dynalog_arrayB[:,0]) * assumed_total_mu

assert np.all(muA == muB)

mu = muA

np.shape(mu)

In [ ]:
# mlc_1

In [ ]:
mlcs = np.swapaxes(np.array([mlc_A, mlc_B]).T, 0, 1)
mlcs = mlcs
np.shape(mlcs)

In [ ]:
# for i in range(60):
#     plt.figure()
#     plt.plot(mlc_A[:,i])
#     plt.plot(-mlc_B[:,i])

#     plt.title(i)
#     plt.show()

In [ ]:
leaf_widths = np.array([10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,  5.,  5.,  5.,
        5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,
        5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,
        5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5., 10., 10.,
       10., 10., 10., 10., 10., 10., 10., 10.])

In [ ]:
mu_density = calc_mu_density(mu, mlcs, jaw, leaf_pair_widths=leaf_widths)

In [ ]:
grid = get_grid(leaf_pair_widths=leaf_widths)

In [ ]:
plt.figure(figsize=(10,8.5))
plt.pcolormesh(grid['mlc'][-1::-1], grid['jaw'][-1::-1], mu_density)
plt.colorbar()
plt.xlabel('MLCX direction (mm)')
plt.ylabel('ASYMY travel direction (mm)')

plt.axis('equal')